In [1]:
#Importing library

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from scipy import stats
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
df = pd.read_csv("dataset/train_processed.csv")

In [3]:
df_date=df['Date']
df_id=df['StoreID']
df_no_date=df.drop(['Date'], axis=1)
df_no_date=df.drop(['Date','StoreID'], axis=1)
df_norm= (df_no_date - df_no_date.min()) / (df_no_date.max() - df_no_date.min())
df=df_norm
df['Date']=df_date
df['StoreID']=df_id
df.head(5)

,Unnamed: 0,IsHoliday,IsOpen,HasPromotions,StoreType,NearestCompetitor,NumberOfCustomers,NumberOfSales,Region_AreaKM2,Region_GDP,...,IsSaturday,IsSunday,WasOpenYesterday,IsOpenTomorrow,YesterdaySales,Previous3DaysSales,PreviousWeekSales,PreviousMonthSales,Date,StoreID
0,0.000000,0.0,1.0,1.0,0.666667,0.003281,0.383500,0.446943,0.291715,0.515529,...,0.0,0.0,1.0,0.0,0.339890,0.363301,0.355788,0.372606,2016-03-31,1000
1,0.000002,1.0,0.0,1.0,0.666667,0.003281,0.000000,0.000000,0.291715,0.515529,...,0.0,0.0,1.0,1.0,0.446943,0.409861,0.384367,0.384941,2016-04-01,1000
2,0.000004,0.0,1.0,0.0,0.666667,0.003281,0.385766,0.417740,0.291715,0.515529,...,1.0,0.0,0.0,0.0,0.000000,0.290735,0.326042,0.368885,2016-04-02,1000
3,0.000006,0.0,0.0,0.0,0.666667,0.003281,0.000000,0.000000,0.291715,0.515529,...,0.0,1.0,1.0,0.0,0.417740,0.319501,0.363920,0.377885,2016-04-03,1000
4,0.000008,1.0,0.0,0.0,0.666667,0.003281,0.000000,0.000000,0.291715,0.515529,...,0.0,0.0,0.0,1.0,0.000000,0.154355,0.363920,0.362002,2016-04-04,1000


In [4]:
df_val3=df.loc[(df['Date']>='2018-01-01')]
df_val3_rest=df.loc[(df['Date']<'2018-01-01')]
df_val2=df_val3_rest.loc[(df_val3_rest['Date']>='2017-11-01')]
df_val2_rest=df.loc[(df['Date']<'2017-11-01')]
df_val1=df_val2_rest.loc[(df_val2_rest['Date']>='2017-09-01')]
df_train=df.loc[(df['Date']<'2017-09-01')]

In [5]:
df_train_clear=df_train.drop(['Date'], axis=1)
df_val1_clear=df_val1.drop(['Date'], axis=1)
df_val2_clear=df_val2.drop(['Date'], axis=1)
df_val3_clear=df_val3.drop(['Date'], axis=1)

In [6]:
y_alldt=df_train_clear[(['StoreID','NumberOfSales'])]
y_val1_alldt=df_val1_clear[(['StoreID','NumberOfSales'])]
y_val2_alldt=df_val2_clear[(['StoreID','NumberOfSales'])]
y_val3_alldt=df_val3_clear[(['StoreID','NumberOfSales'])]

In [7]:
X_alldt=df_train_clear.drop(['Unnamed: 0','NumberOfSales','NumberOfCustomers','YesterdaySales','Previous3DaysSales','PreviousWeekSales','PreviousMonthSales'], axis=1)
X_val1_alldt=df_val1_clear.drop(['Unnamed: 0','NumberOfSales','NumberOfCustomers','YesterdaySales','Previous3DaysSales','PreviousWeekSales','PreviousMonthSales'], axis=1)
X_val2_alldt=df_val2_clear.drop(['Unnamed: 0','NumberOfSales','NumberOfCustomers','YesterdaySales','Previous3DaysSales','PreviousWeekSales','PreviousMonthSales'], axis=1)
X_val3_alldt=df_val3_clear.drop(['Unnamed: 0','NumberOfSales','NumberOfCustomers','YesterdaySales','Previous3DaysSales','PreviousWeekSales','PreviousMonthSales'], axis=1)

print (X_alldt.head(5))
print (y_alldt.head(5))
print( X_alldt['StoreID'].unique())

In [8]:
X=X_alldt.loc[(X_alldt['StoreID']==1006)]
y=y_alldt.loc[(y_alldt['StoreID']==1006)]
X_val1=X_val1_alldt.loc[(X_val1_alldt['StoreID']==1006)]
y_val1=y_val1_alldt.loc[(y_val1_alldt['StoreID']==1006)]

In [9]:
X=X.drop(['StoreID'], axis=1)
y=y.drop(['StoreID'], axis=1)
X_val1=X_val1.drop(['StoreID'], axis=1)
y_val1=y_val1.drop(['StoreID'], axis=1)

In [10]:
y=y['NumberOfSales']
y_val1=y_val1['NumberOfSales']
print(X.shape)
print(y.shape)
print(X_val1.shape)
print(y_val1.shape)


(519, 40)
(519,)
(61, 40)
(61,)


In [11]:
columns=['num_of_param','num_of_tree','depth','mse']
index=range(1,2500)
hyp_par=pd.DataFrame(index=index,columns=columns)

In [12]:
count=0
min=1
depth=25
nt=33
for na in range(1,41,1):
    forest = RandomForestRegressor(max_depth=depth, random_state=0, n_estimators=nt, max_features=na, n_jobs=-1)
    forest.fit(X, y)
    y_pred=forest.predict(X_val1)
    hyp_par.at[count,'num_of_param']=na
    hyp_par.at[count,'mse']=mean_squared_error(y_val1,y_pred)
    count+=1

In [13]:
hyp_par_sort=hyp_par.sort_values(['mse'])
best10=hyp_par_sort.head(10)
best10=best10['num_of_param'].astype(int)
best10A=list(best10.values)
best10A

[25, 12, 18, 15, 10, 13, 9, 24, 14, 17]

In [14]:
count=0
min=1
depth=25
for na in best10A:
    for nt in range(1,250,10):
        forest = RandomForestRegressor(max_depth=depth, random_state=0, n_estimators=nt, max_features=na, n_jobs=-1)
        forest.fit(X, y)
        y_pred=forest.predict(X_val1)
        hyp_par.at[count,'num_of_param']=na
        hyp_par.at[count,'num_of_tree']=nt
        hyp_par.at[count,'mse']=mean_squared_error(y_val1,y_pred)
        count+=1
    print(na)

25
12
18
15
10
13
9
24
14
17


In [15]:
hyp_par_sort=hyp_par.sort_values(['mse'])
best10=hyp_par_sort.head(10)
best10A=best10['num_of_param'].astype(int)
best10A=list(best10A.values)
best10B=best10['num_of_tree'].astype(int)
best10B=list(best10B.values)
print(best10A)
print(best10B)

[14, 25, 9, 15, 25, 25, 15, 15, 25, 9]
[11, 21, 21, 101, 31, 41, 111, 121, 11, 11]


In [16]:
count=0
min=1
for num in range(0,10,1):
    for depth in range(1,100,5):
        forest = RandomForestRegressor(max_depth=depth, random_state=0, n_estimators=best10B[num], max_features=best10A[num], n_jobs=-1)
        forest.fit(X, y)
        y_pred=forest.predict(X_val1)
        hyp_par.at[count,'num_of_param']=best10A[num]
        hyp_par.at[count,'num_of_tree']=best10B[num]
        hyp_par.at[count,'depth']=depth
        hyp_par.at[count,'mse']=mean_squared_error(y_val1,y_pred)
        count+=1
    print(num)

0
1
2
3
4
5
6
7
8
9


In [17]:
hyp_par_sort=hyp_par.sort_values(['mse'])
hyp_par_sort.head(10)

,num_of_param,num_of_tree,depth,mse
7,14,11,36,0.000855137
201,14,11,NaN,0.000855137
11,14,11,56,0.000855137
8,14,11,41,0.000855137
14,14,11,71,0.000855137
12,14,11,61,0.000855137
19,14,11,96,0.000855137
5,14,11,26,0.000855137
13,14,11,66,0.000855137
6,14,11,31,0.000855137
